In [22]:
%tensorflow_version 2.x #specifying the tf version i want to use


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [39]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

import random
from tensorflow.keras.layers import Flatten,Conv2D,Dense,MaxPooling2D
from tensorflow.keras.optimizers import SGD


In [26]:
# The MNIST Fashion dataset is installed with keras read more about the dataset at:https://www.kaggle.com/datasets/zalando-research/fashionmnist
fashion_mnist = keras.datasets.fashion_mnist # loading the dataset containing images
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()


In [27]:
train_images = train_images.reshape((train_images.shape[0],28,28,1))
test_images = test_images.reshape((test_images.shape[0],28,28,1))


When dealing with images, we need a tensor with 4 dimensions: batch size, width, height, and color channels.

x_train is (60000, 28, 28). We need to reshape it to add the missing dimension ("1" because these images are grayscale.)

In [28]:
train_images = train_images.reshape((train_images.shape[0],28,28,1))

Each pixel goes from 0 to 255. Neural networks work much better with smaller values.

Here we normalize pixels by dividing them by 255. That way, each pixel will go from 0 to 1.

In [41]:
train_images = train_images / 255.0



This line one-hot encodes these values.

For example, this will transform a value like 5, in an array of zeros with a single 1 corresponding to the fifth index:

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [30]:
train_labels = keras.utils.to_categorical(train_labels)

Let's now define our model.There are several ways to create a model in Keras. This one is called the "Sequential API". Our model will be a sequence of layers that we will define one by one.

A lot is going on with this first line.First, we define our model's input shape: a 28x28x1 tensor (width, height, channels.).This is exactly the shape we have in our train dataset.

Then we define our first layer: a Conv2D layer with 32 filters and a 3x3 kernel.This layer will generate 32 different representations using the training images. We also need to define the activation function used for this layer: ReLU.You'll see ReLU everywhere. It's a popular activation function.It will allow us to solve non-linear problems, like recognizing handwritten digits (the train_labes and test_labels).

After our Conv2D layer, we have a max pooling operation.The goal of this layer is to downsample the amount of information collected by the convolutional layer.We want to throw away unimportant details and retain what truly matters.

We are now going to flatten the output. We want everything in a continuous list of values.That's what the Flatten layer does. It will give us a flat tensor.

Finally, we have a couple of Dense layers. Notice how the output layer has a size of 10, one for each of our possible digit values, and a softmax activation. The softmax ensures we get a probability distribution indicating the most likely digit in the image.


In [33]:
#Buiding the model
model = keras.models.Sequential ([
    Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(100,activation='relu'),
    Dense(10,activation='softmax')

    
])

After creating our model, we compile it.I'm using Stochastic Gradient Descent (SGD) as the optimizer.The loss is categorical cross-entropy: this is a multi-class classification problem.We want to record the accuracy as the model trains.

In [34]:
#compling the model
optimizer = SGD(learning_rate=0.01,momentum=0.0)
model.compile(
    optimizer=optimizer,
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

Finally, we fit the model. This starts training it.A couple of notes:

• I'm using a batch size of 32 images.
• I'm running 10 total epochs.

When fit() is done, we have a fully trained model!


In [ ]:
history = model.fit(train_images,train_labels, epochs=10, batch_size=32)

This gets a random image from the test set and displays it.
Notice that we want the image to come from the test set, containing data the model didn't see during training.

In [ ]:
image = random.choice(test_images)
plt.imshow(image, cmap=plt.get_cmap('gray'))
plt.show

We can't forget to reshape and normalize the image as we did before with the entire train set.

I'm doing it this time for the image I use to test the model.

In [43]:
image = (image.reshape((1,28,28,1))).astype('float32') /255.0

Finally, I predict the value of the image.

Remember that the result is a one-hot-encoded vector. That's why I take the argmax value (the position with the highest probability) and that's the result

In [44]:
digit = np.argmax(model.predict(image)[0],axis=-1)
print("prediction:",digit)

prediction: 5
